# Extracting WGS Variant Data from All of Us

This tutorial demonstrates how to extract short-read Whole Genome Sequencing (srWGS) data from the All of Us Research Program using Hail.

**Workflow Overview:**
1. Environment setup and Hail initialization
2. Load the WGS MatrixTable
3. Filter variants by genomic intervals (specific SNPs)
4. Convert genotypes to allele counts
5. Export to CSV for downstream analysis

**Prerequisites:**
- Controlled Tier access in All of Us
- Workspace with WGS data access enabled

---

# Environment Setup

## Check CDR Version

Document the CDR version for reproducibility.

In [1]:
# Display current CDR version
!echo $WORKSPACE_CDR

fc-aou-cdr-prod-ct.C2022Q4R13


## Configure Workspace Bucket

The workspace bucket stores intermediate and output files.

In [2]:
import os

bucket = os.getenv("WORKSPACE_BUCKET")
bucket

'gs://fc-secure-28df46b0-6f9d-4443-ae5f-cb0492e90c24'

## Initialize Hail

Hail is a genomic data analysis library built on Apache Spark. We set GRCh38 as the default reference genome (used by All of Us).

In [3]:
import hail as hl

# Initialize Hail with GRCh38 reference genome
hl.default_reference(new_default_reference="GRCh38")

Loading BokehJS ...

Initializing Hail with default parameters...
/opt/conda/lib/python3.10/site-packages/hailtop/aiocloud/aiogoogle/user_config.py:43: UserWarning:

Reading spark-defaults.conf to determine GCS requester pays configuration. This is deprecated. Please use `hailctl config set gcs_requester_pays/project` and `hailctl config set gcs_requester_pays/buckets`.

Running on Apache Spark version 3.5.3
SparkUI available at http://all-of-us-28664-m.us-central1-c.c.terra-vpc-sc-39ac9e8b.internal:33473
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.134-952ae203dbbe
LOGGING: writing to /home/jupyter/workspaces/multimodeltestzhiyu/hail-20251220-1003-0.2.134-952ae203dbbe.log
2025-12-20 10:05:00.432 Hail: INFO: wrote matrix table with 19 rows and 245394 columns in 19 partitions to mt_filtered.mt
2025-12-20 10:05:02.985 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
    To preserve row-major matrix table order, firs

---
# Load WGS MatrixTable

The WGS data is stored as a Hail MatrixTable. We access it via environment variables that point to the controlled-tier data.

## Get MatrixTable Path

All of Us provides several WGS data formats. Here we use the ACAF threshold split MatrixTable.

In [4]:
# Get the path to the WGS MatrixTable from environment variable
mt_wgs_path = os.getenv("WGS_ACAF_THRESHOLD_SPLIT_HAIL_PATH")
mt_wgs_path

'gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/splitMT/hail.mt'

## Read the MatrixTable

In [5]:
# Load the MatrixTable
mt = hl.read_matrix_table(mt_wgs_path)

## Explore MatrixTable Structure

Understanding the schema helps us know what fields are available for filtering and extraction.

In [6]:
# Display MatrixTable schema
# - Row fields: variant-level annotations (locus, alleles, QC metrics)
# - Column fields: sample-level annotations (person_id)
# - Entry fields: genotype data (GT, GQ, DP, AD)
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'filters': set<str>
    'a_index': int32
    'was_split': bool
    'variant_qc': struct {
        gq_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        call_rate: float64, 
        n_called: int64, 
        n_not_called: int64, 
        n_filtered: int64, 
        n_het: int64, 
        n_non_ref: int64, 
        het_freq_hwe: float64, 
        p_value_hwe: float64, 
        p_value_excess_het: float64
    }
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        homozygote_count: array<int32>
    }
----------------------------------------
Entry fields:
    'GT': call
    'GQ': int32
    'RGQ': int32
    '

## Check MatrixTable Dimensions

Count the number of variants (rows) and samples (columns).

In [7]:
# Returns (n_variants, n_samples)
mt.count()

(99250816, 245394)

---
# Filter Variants by Genomic Intervals

Rather than working with the full genome, we filter to specific regions of interest (e.g., SNPs from a GWAS study).

## (Optional) Filter by Person ID

If you have a specific cohort, you can filter columns first to reduce computation.

In [8]:
# Uncomment to filter to specific participants
# keep_ids = ['1166759']  # List of person_ids
# mt = mt.filter_cols(hl.literal(keep_ids).contains(mt.s))

## Define Genomic Intervals

Specify the genomic regions containing your SNPs of interest. Format: `chr:start-end`

In [9]:
# Define intervals for SNPs of interest
# These are example intervals - replace with your target regions
test_intervals = [
    'chr14:104920174-104920175',
    'chr6:159082054-159082055',
    'chr14:68287978-68287979',
    'chr6:36414159-36414160',
    'chr1:116738074-116738075',
    'chr5:143224856-143224857',
    'chr9:34710263-34710264',
    'chr13:39781776-39781777',
    'chr12:111446804-111446805',
    'chr12:45976333-45976334',
    'chr8:128564231-128564232',
    'chr18:3542249-3542250',
    'chr18:12880207-12880208',
    'chr19:35722170-35722171',
    'chr15:69699078-69699079',
    'chr9:34710263-34710264',
    'chr4:48218822-48218823',
    'chr1:2552522-2552523',
    'chr5:134503843-134503844',
    'chr15:90361626-90361627',
]

## Filter MatrixTable to Intervals

In [10]:
# Parse interval strings to Hail interval objects
interval_exprs = [hl.parse_locus_interval(iv, reference_genome='GRCh38') 
                  for iv in test_intervals]

# Filter MatrixTable to only include variants within these intervals
mt_subset = hl.filter_intervals(mt, interval_exprs)

In [11]:
# Check how many variants remain after filtering
mt_subset.count()

(23, 245394)

In [12]:
# Preview the filtered variants
mt_subset.rows().show()

+----------------+------------+----------+---------+-----------+
| locus          | alleles    | filters  | a_index | was_split |
+----------------+------------+----------+---------+-----------+
| locus<GRCh38>  | array<str> | set<str> |   int32 |      bool |
+----------------+------------+----------+---------+-----------+
| chr1:2552522   | ["G","A"]  | {}       |       1 |      True |
| chr1:2552522   | ["G","C"]  | {}       |       2 |      True |
| chr1:116738074 | ["C","T"]  | {}       |       1 |     False |
| chr4:48218822  | ["G","A"]  | {}       |       1 |     False |
| chr5:134503843 | ["C","T"]  | {}       |       1 |     False |
| chr5:143224856 | ["A","G"]  | {}       |       1 |     False |
| chr6:36414159  | ["G","GA"] | {}       |       1 |     False |
| chr6:159082054 | ["A","G"]  | {}       |       1 |     False |
| chr8:128564231 | ["T","C"]  | {}       |       1 |     False |
| chr9:34710263  | ["G","A"]  | {}       |       1 |     False |
+----------------+------------+----------+---------+-----------+

+--------------------------+---------------------------+
| variant_qc.gq_stats.mean | variant_qc.gq_stats.stdev |
+--------------------------+---------------------------+
|                  float64 |                   float64 |
+--------------------------+---------------------------+
|                 5.19e+01 |                  2.04e+01 |
|                 5.19e+01 |                  2.04e+01 |
|                 7.16e+01 |                  2.58e+01 |
|                 5.27e+01 |                  1.97e+01 |
|                 4.55e+01 |                  1.32e+01 |
|                 5.75e+01 |                  2.27e+01 |
|                 7.34e+01 |                  2.57e+01 |
|                 7.37e+01 |                  2.54e+01 |
|                 4.64e+01 |                  1.49e+01 |
|                 6.79e+01 |                  2.58e+01 |
+--------------------------+---------------------------+

+-------------------------+-------------------------+----------------------+
| variant_qc.gq_stats.min | variant_qc.gq_stats.max | variant_qc.call_rate |
+-------------------------+-------------------------+----------------------+
|                 float64 |                 float64 |              float64 |
+-------------------------+-------------------------+----------------------+
|                3.00e+00 |                9.90e+01 |             1.00e+00 |
|                3.00e+00 |                9.90e+01 |             1.00e+00 |
|                2.00e+00 |                9.90e+01 |             1.00e+00 |
|                3.00e+00 |                9.90e+01 |             1.00e+00 |
|                5.00e+00 |                9.90e+01 |             1.00e+00 |
|                2.00e+00 |                9.90e+01 |             1.00e+00 |
|                1.00e+00 |                9.90e+01 |             1.00e+00 |
|                3.00e+00 |                9.90e+01 |             1.00e+00 |
|                6.00e+00 |                9.90e+01 |             1.00e+00 |
|                5.00e+00 |                9.90e+01 |             1.00e+00 |
+-------------------------+-------------------------+----------------------+

+---------------------+-------------------------+-----------------------+
| variant_qc.n_called | variant_qc.n_not_called | variant_qc.n_filtered |
+---------------------+-------------------------+-----------------------+
|               int64 |                   int64 |                 int64 |
+---------------------+-------------------------+-----------------------+
|              245374 |                       1 |                    19 |
|              245374 |                       1 |                    19 |
|              245289 |                       0 |                   105 |
|              245381 |                       0 |                    13 |
|              245382 |                       0 |                    12 |
|              245385 |                       0 |                     9 |

---
# Filter to Specific SNPs

Further refine to exact SNP positions using SNP IDs (chr:pos:ref:alt format).

## Create SNP Identifiers

In [13]:
# Create a SNP ID column for easier filtering
# Format: chromosome:position:ref:alt (e.g., "14:104920174:G:A")
mt_subset = mt_subset.annotate_rows(
    snp_id=mt_subset.locus.contig.replace("chr", "") + ":" + hl.str(mt_subset.locus.position) + 
           ":" + mt_subset.alleles[0] + ":" + mt_subset.alleles[1]
)

## Define Target SNPs

In [14]:
# List of SNP IDs to extract (format: chr:pos:ref:alt)
snp_ids = [
    "14:104920174:G:A", "6:159082054:A:G", "14:68287978:G:A",
    "6:36414159:G:GA", "13:39781776:T:C", "12:45976333:C:G",
    "12:111446804:T:C", "9:34710263:G:A", "5:143224856:A:G",
    "1:116738074:C:T", "8:128564231:T:C", "18:3542249:T:C",
    "18:12880207:A:G", "19:35722170:A:G", "15:69699078:G:A",
    "9:34710263:G:A", "4:48218822:G:A", "1:2552522:G:A",
    "5:134503843:C:T", "15:90361626:G:A"
]

## Filter to Target SNPs

In [15]:
# Create a set for efficient lookup
snp_set = hl.set(snp_ids)

# Filter to only keep rows where snp_id is in our target list
mt_filtered = mt_subset.filter_rows(snp_set.contains(mt_subset.snp_id))

# Checkpoint to materialize the filtered data (improves performance for downstream operations)
mt_filtered = mt_filtered.checkpoint("mt_filtered.mt", overwrite=True)

In [16]:
# Verify the number of SNPs after filtering
mt_filtered.count()

(19, 245394)

In [17]:
# Preview the filtered variants
mt_filtered.rows().show()

+----------------+------------+----------+---------+-----------+
| locus          | alleles    | filters  | a_index | was_split |
+----------------+------------+----------+---------+-----------+
| locus<GRCh38>  | array<str> | set<str> |   int32 |      bool |
+----------------+------------+----------+---------+-----------+
| chr1:2552522   | ["G","A"]  | {}       |       1 |      True |
| chr1:116738074 | ["C","T"]  | {}       |       1 |     False |
| chr4:48218822  | ["G","A"]  | {}       |       1 |     False |
| chr5:134503843 | ["C","T"]  | {}       |       1 |     False |
| chr5:143224856 | ["A","G"]  | {}       |       1 |     False |
| chr6:36414159  | ["G","GA"] | {}       |       1 |     False |
| chr6:159082054 | ["A","G"]  | {}       |       1 |     False |
| chr8:128564231 | ["T","C"]  | {}       |       1 |     False |
| chr9:34710263  | ["G","A"]  | {}       |       1 |     False |
| chr12:45976333 | ["C","G"]  | {}       |       1 |      True |
+----------------+------------+----------+---------+-----------+

+--------------------------+---------------------------+
| variant_qc.gq_stats.mean | variant_qc.gq_stats.stdev |
+--------------------------+---------------------------+
|                  float64 |                   float64 |
+--------------------------+---------------------------+
|                 5.19e+01 |                  2.04e+01 |
|                 7.16e+01 |                  2.58e+01 |
|                 5.27e+01 |                  1.97e+01 |
|                 4.55e+01 |                  1.32e+01 |
|                 5.75e+01 |                  2.27e+01 |
|                 7.34e+01 |                  2.57e+01 |
|                 7.37e+01 |                  2.54e+01 |
|                 4.64e+01 |                  1.49e+01 |
|                 6.79e+01 |                  2.58e+01 |
|                 8.13e+01 |                  2.39e+01 |
+--------------------------+---------------------------+

+-------------------------+-------------------------+----------------------+
| variant_qc.gq_stats.min | variant_qc.gq_stats.max | variant_qc.call_rate |
+-------------------------+-------------------------+----------------------+
|                 float64 |                 float64 |              float64 |
+-------------------------+-------------------------+----------------------+
|                3.00e+00 |                9.90e+01 |             1.00e+00 |
|                2.00e+00 |                9.90e+01 |             1.00e+00 |
|                3.00e+00 |                9.90e+01 |             1.00e+00 |
|                5.00e+00 |                9.90e+01 |             1.00e+00 |
|                2.00e+00 |                9.90e+01 |             1.00e+00 |
|                1.00e+00 |                9.90e+01 |             1.00e+00 |
|                3.00e+00 |                9.90e+01 |             1.00e+00 |
|                6.00e+00 |                9.90e+01 |             1.00e+00 |
|                5.00e+00 |                9.90e+01 |             1.00e+00 |
|                7.00e+00 |                9.90e+01 |             1.00e+00 |
+-------------------------+-------------------------+----------------------+

+---------------------+-------------------------+-----------------------+
| variant_qc.n_called | variant_qc.n_not_called | variant_qc.n_filtered |
+---------------------+-------------------------+-----------------------+
|               int64 |                   int64 |                 int64 |
+---------------------+-------------------------+-----------------------+
|              245374 |                       1 |                    19 |
|              245289 |                       0 |                   105 |
|              245381 |                       0 |                    13 |
|              245382 |                       0 |                    12 |
|              245385 |                       0 |                     9 |
|              245391 |                       0 |                     3 |

---
# Convert Genotypes to Allele Counts

Transform diploid genotype calls (GT) into numeric allele counts for statistical analysis:
- Homozygous reference (0/0) → 0
- Heterozygous (0/1) → 1  
- Homozygous alternate (1/1) → 2
- Missing → NA

In [18]:
# Annotate entries with allele count
mt_snps = mt_filtered.annotate_entries(
    allele_count=hl.case()
        .when(mt_filtered.GT.is_hom_ref(), 0)   # Homozygous reference → 0 alt alleles
        .when(mt_filtered.GT.is_het(), 1)       # Heterozygous → 1 alt allele
        .when(mt_filtered.GT.is_hom_var(), 2)   # Homozygous alternate → 2 alt alleles
        .or_missing()                           # Missing data remains missing
)

---
# Export to Pandas DataFrame

Convert the Hail MatrixTable to a wide-format pandas DataFrame for downstream analysis.

## Extract and Reshape Data

In [19]:
# Convert to entries table and select relevant columns
table = mt_snps.entries()
table = table.key_by()
table = table.select('s', 'snp_id', 'allele_count')

# Pivot to wide format: rows = individuals, columns = SNPs
snp_matrix = table.to_pandas().pivot(index="s", columns="snp_id", values="allele_count")

## Clean Up Column Names

In [20]:
# Reset index and rename the sample column to person_id
snp_matrix = snp_matrix.reset_index().rename(columns={"s": "person_id"})

# Convert person_id to integer
snp_matrix['person_id'] = snp_matrix['person_id'].astype(int)

## Save Locally

In [21]:
# Save to local CSV
snp_matrix.to_csv("extract_gene_test.csv", index=False)

In [22]:
# Verify the output
import pandas as pd
df = pd.read_csv('extract_gene_test.csv')
df.shape

(245394, 20)

---
# Upload to Google Cloud Storage

Save the output to your workspace bucket for persistent storage and sharing.

In [23]:
import os
import subprocess
import numpy as np
import pandas as pd

In [24]:
# Configuration
my_dataframe = df                        # DataFrame to upload
destination_filename = 'RA_wgs.csv'      # Name for the file in bucket

# ============================================================
# DO NOT MODIFY BELOW THIS LINE
# ============================================================

# Save DataFrame locally
my_dataframe.to_csv(destination_filename, index=False)

# Get workspace bucket
my_bucket = os.getenv('WORKSPACE_BUCKET')

# Upload to bucket using gsutil
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# Print result
output.stderr

b'Copying file://./RA_wgs.csv [Content-Type=text/csv]...\n/ [0 files][    0.0 B/ 19.2 MiB]                                                \r/ [1 files][ 19.2 MiB/ 19.2 MiB]                                                \r-\r\nOperation completed over 1 objects/19.2 MiB.                                     \n'

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
    self._run_once()
  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 1894, in _run_once
    handle = self._ready.popleft()
IndexError: pop from 

---
# Summary

This notebook demonstrated how to:

1. **Initialize Hail** with GRCh38 reference genome
2. **Load WGS data** from the All of Us controlled tier
3. **Filter variants** by genomic intervals and specific SNP IDs
4. **Convert genotypes** to allele counts (0/1/2 encoding)
5. **Export data** to CSV and upload to cloud storage

**Output:** A CSV file with rows = participants, columns = SNPs, values = allele counts (0, 1, or 2)

**Next Steps:**
- Merge with phenotype data for association analysis
- Perform quality control on the extracted variants
- Run statistical analyses (e.g., logistic regression, PRS calculation)